# Understanding Combinatorial Explosion in Materials Science with SMACT

## Introduction
 Materials discovery involves exploring vast chemical spaces of possible element combinations. This tutorial will demonstrate how to:

 1. Calculate raw combinatorial possibilities
 2. Apply chemical filters to reduce the search space
 3. Generate practical composition lists for further analysis


## Part 1: Basic Combinatorics & Stoichiometric Constraints
First, let's understand the raw scale of possible combinations - This section of the notebook aims to demonstrate the sheer scale of the combinatorial explosion. Based on some assumtions about how many elements (103) or species - elements in oxidation states - (403) there are, it calculates how many binary, ternary, quaternary... combinations there can be, within different integers stoichiometry limits.

this section contains **ZERO** chemistry. rather it is a mathematical exercise to demonstrate the claims above.

The numbers that are produced match roughly to the raw combinations (before chemical filters) numbers obtained in Table 1 of the publication [Computational Screening of All Stoichiometric Inorganic Materials](https://www.cell.com/chem/fulltext/S2451-9294(16)30155-3) (D. W. Davies et. al, Chem, 2016).
Small deviations from the numbers are due to changes in what we class as an "accessible" oxidation state for some elements. **- to do reference this in the jupyter-books**

N.B. It can take a minute or two to calculate quaternaries on a desktop/laptop computer. 


In [ ]:
import itertools
from math import gcd

def count_combinations(n_elements, search_space=403):
    """Count unique combinations of n elements from the periodic table"""
    return sum(1 for _ in itertools.combinations(range(search_space), n_elements))

def is_irreducible(stoichs):
    """Check if stoichiometry can be reduced to smaller integers"""
    for i in range(1, len(stoichs)):
        if gcd(stoichs[i-1], stoichs[i]) == 1:
            return True
    return False

def count_stoichiometries(n_elements, max_coefficient=8):
    """Count possible stoichiometric ratios for n elements up to max_coefficient"""
    count = sum(1 for combo in itertools.product(
        *(n_elements * (tuple(range(1, max_coefficient + 1)),))
    ) if is_irreducible(combo))
    return count

def main(search_space):
    """Calculate combinations and stoichiometries for a given search space"""
    compound_names = {2: "binary", 3: "ternary", 4: "quaternary"}
    
    print(f"\nIn a search space of {search_space} {'elements' if search_space==103 else 'element-oxidation states'}:")
    
    # Calculate element combinations
    combinations = {}
    for n in range(2, 5):
        count = count_combinations(n, search_space)
        combinations[n] = count
        print(f"Number of unique combinations of {n} elements: {count}")
    
    print("\nStoichiometry calculations:")
    # Calculate stoichiometries for different max coefficients
    for n in range(2, 5):
        for max_coeff in [2, 4, 6, 8]:
            stoich_count = count_stoichiometries(n, max_coeff)
            total = combinations[n] * stoich_count
            print(f"Unique {compound_names[n]} compounds with max coefficient {max_coeff}: {total:5.3e}")
            print(f"(from {combinations[n]} combinations × {stoich_count} stoichiometries)")

if __name__ == "__main__":
    # Run for both elements and oxidation states
    print("=== Calculations for elements only ===")
    main(103)
    
    print("\n=== Calculations for elements in oxidation states ===")
    main(403)

=== Calculations for elements only ===

In a search space of 103 elements:
Number of unique combinations of 2 elements: 5253
Number of unique combinations of 3 elements: 176851
Number of unique combinations of 4 elements: 4421275

Stoichiometry calculations:
Unique binary compounds with max coefficient 2: 1.576e+04
(from 5253 combinations × 3 stoichiometries)
Unique binary compounds with max coefficient 4: 5.778e+04
(from 5253 combinations × 11 stoichiometries)
Unique binary compounds with max coefficient 6: 1.208e+05
(from 5253 combinations × 23 stoichiometries)
Unique binary compounds with max coefficient 8: 2.259e+05
(from 5253 combinations × 43 stoichiometries)
Unique ternary compounds with max coefficient 2: 1.238e+06
(from 176851 combinations × 7 stoichiometries)
Unique ternary compounds with max coefficient 4: 9.727e+06
(from 176851 combinations × 55 stoichiometries)
Unique ternary compounds with max coefficient 6: 3.130e+07
(from 176851 combinations × 177 stoichiometries)
Uniqu

## Part 2: Adding Stoichiometry Constraints
Now we'll consider different possible ratios between elements:

In [ ]:
def count_stoichiometries(n_elements, max_coefficient=8):
    """Count possible stoichiometric ratios for n elements up to max_coefficient"""
    def is_irreducible(stoichs):
        """Check if stoichiometry can be reduced to smaller integers"""
        for i in range(1, len(stoichs)):
            if gcd(stoichs[i - 1], stoichs[i]) == 1:
                return True
        return False
    
    count = sum(1 for combo in itertools.product(
        *(n_elements * (tuple(range(1, max_coefficient + 1)),))
    ) if is_irreducible(combo))
    
    return count

# Example usage
n_elements = 3  # ternary compounds
max_coeff = 8
count = count_stoichiometries(n_elements, max_coeff)
print(f"Number of possible stoichiometries: {count}")

Number of possible stoichiometries: 433


In [ ]:
from smact import Element, element_dictionary, ordered_elements
from smact.screening import smact_filter
import multiprocessing
from datetime import datetime

def generate_viable_compositions(elements, n_elements=3):
    """Generate chemically viable compositions using SMACT filters"""
    # Create element combinations
    element_combos = itertools.combinations(elements, n_elements)
    systems = [[*combo] for combo in element_combos]
    
    # Apply SMACT filtering in parallel
    with multiprocessing.Pool(processes=4) as pool:
        results = pool.map(smact_filter, systems)
    
    # Flatten results
    compositions = [item for sublist in results for item in sublist]
    return compositions

# Example usage
# Get first row transition metals
elements = [Element(symbol) for symbol in ordered_elements(21, 30)]
compositions = generate_viable_compositions(elements)
print(f"Number of viable compositions: {len(compositions)}")

Number of viable compositions: 18696


In [ ]:
import pandas as pd
from pymatgen.core import Composition

def format_compositions(compositions):
    """Convert SMACT compositions to readable formulas"""
    def comp_to_formula(comp):
        formula = "".join(f"{el}{amt}" for el, amt in zip(comp[0], comp[2]))
        return Composition(formula).reduced_formula
    
    formulas = [comp_to_formula(comp) for comp in compositions]
    return pd.DataFrame({"formula": formulas}).drop_duplicates()

# Example usage
df = format_compositions(compositions)
print("\nSample of viable compositions:")
print(df.head())


Sample of viable compositions:
   formula
0   Sc2TiV
1  Sc3TiV2
2  Sc3Ti2V
3  Sc4TiV3
4  Sc4Ti3V
